In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import interpolate
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
#this next line is only needed in iPython notebooks
%matplotlib inline 
import math
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
font = fm.FontProperties(size = 12)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
def hide_spines(intx=False,inty=False):
    """Hides the top and rightmost axis spines from view for all active
    figures and their respective axes."""

    # Retrieve a list of all current figures.
    figures = [x for x in matplotlib._pylab_helpers.Gcf.get_all_fig_managers()]
    if (plt.gca().get_legend()):
        plt.setp(plt.gca().get_legend().get_texts(), fontproperties=font) 
    for figure in figures:
        # Get all Axis instances related to the figure.
        for ax in figure.canvas.figure.get_axes():
            # Disable spines.
            ax.spines['right'].set_color('none')
            ax.spines['top'].set_color('none')
            # Disable ticks.
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')
           # ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
            for label in ax.get_xticklabels() :
                label.set_fontproperties(font)
            for label in ax.get_yticklabels() :
                label.set_fontproperties(font)
            #ax.set_xticklabels(ax.get_xticks(), fontproperties = font)
            ax.set_xlabel(ax.get_xlabel(), fontproperties = font)
            ax.set_ylabel(ax.get_ylabel(), fontproperties = font)
            ax.set_title(ax.get_title(), fontproperties = font)
            if (inty):
                ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
            if (intx):
                ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
def show(nm,a=0,b=0):
    hide_spines(a,b)
    #ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
    #plt.yticks([1,1e-2,1e-4,1e-6,1e-8,1e-10,1e-12], labels)
    #ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
    if (len(nm)>0):
        plt.savefig(nm+".pdf",bbox_inches='tight');
    plt.show()
    
    


%matplotlib inline

In [ ]:
#read in data
alldata = np.loadtxt("FullData.csv", delimiter=",", skiprows=1)
print(alldata[0:2,:])
print(alldata.shape)

In [ ]:
#30 sample points
times = np.linspace(np.min(alldata[:,0]),np.max(alldata[:,0]),30)
print(times)
#plotting points
plt_times = np.linspace(np.min(alldata[:,0]),np.max(alldata[:,0]),200)
#function to wrap interpolation
def interpdata(times, temps, plt_times):
    return interpolate.interp1d(times,temps,bounds_error=False, fill_value="extrapolate",kind="cubic")(plt_times)

In [ ]:
#these are the levels corresponding to ABCDEF
levels = (0.8,0.85,1.05,1.2,1.25)
levels_with1 = (0.8,0.85,1,1.05,1.2,1.25)

levels_dict = {"0.8":"A","0.85":"B","1.0":"C","1.05":"D","1.2":"E","1.25":"F"}
#create matrix
df = np.zeros((5*3+1-1,30)) #subtract 1 because Scale A failed

#fill in the nominal case (1,1,1)=CCC
tmp = alldata[ (alldata[:,3]==1)*(alldata[:,4]==1)*(alldata[:,5]==1),:]
interpfunc = interpolate.interp1d(tmp[:,0],tmp[:,1],bounds_error=False, fill_value="extrapolate")
times = np.linspace(np.min(alldata[:,0]),np.max(alldata[:,0]),30)


df[0,:] = interpfunc(times)
count = 1
settings = np.zeros((5*3+1-1,3))
settings[0,:] = (1,1,1)
tmp = alldata[ (alldata[:,3]==1)*(alldata[:,4]==1)*(alldata[:,5]==1),:]
filename = "BIE-"+ "CCCC" + "-Tr_vs_time.csv"
np.savetxt(X=tmp[:,0:3], fname=filename, delimiter=",")

#now fill in the scale variations
for i in levels:
    if (i > 0.8):
        tmp = alldata[ (alldata[:,3]==i)*(alldata[:,4]==1)*(alldata[:,5]==1),:]
        interpfunc = interpolate.interp1d(tmp[:,0],tmp[:,1],
                                          bounds_error=False, fill_value="extrapolate")
        df[count,:] = interpfunc(times)
        settings[count,:] = (i,1,1)
        count += 1
        scale = levels_dict[str(i)]
        print(scale, i)
        filename = "BIE-"+scale + "CCC" + "-Tr_vs_time.csv"
        np.savetxt(X=tmp[:,0:3], fname=filename, delimiter=",")

        
#now fill in the sc_length variations
for i in levels:
    tmp = alldata[ (alldata[:,3]==1)*(alldata[:,4]==i)*(alldata[:,5]==1),:]
    interpfunc = interpolate.interp1d(tmp[:,0],tmp[:,1],
                                      bounds_error=False, fill_value="extrapolate")
    df[count,:] = interpfunc(times)
    settings[count,:] = (1,i,1)
    count += 1
    sc_length = levels_dict[str(i)]
    filename = "BIE-C" + sc_length + "CC" + "-Tr_vs_time.csv"
    np.savetxt(X=tmp[:,0:3], fname=filename, delimiter=",")

#now fill in the rapt variations
for i in levels:
    tmp = alldata[ (alldata[:,3]==1)*(alldata[:,4]==1)*(alldata[:,5]==i),:]
    interpfunc = interpolate.interp1d(tmp[:,0],tmp[:,1],
                                      bounds_error=False, fill_value="extrapolate")
    df[count,:] = interpfunc(times)
    settings[count,:] = (1,1,i)
    count += 1
    rapt = levels_dict[str(i)]
    filename = "BIE-CC" + rapt + "C" + "-Tr_vs_time.csv"
    np.savetxt(X=tmp[:,0:3], fname=filename, delimiter=",")

In [ ]:
#compute column-wise mean
print(df.shape)
mean_trace = np.mean(df,axis=0)
#plt.plot(times,mean_trace,'bo')
plt.plot(plt_times*10, interpdata(times,mean_trace,plt_times), "b-")
plt.xlabel("time (ns)")
plt.ylabel("$\\bar{T}$ (keV)")
#plt.title("Mean Dante Response of the Data Set")
show("mean_resp")

In [ ]:
df_adj = df-mean_trace
U, s, V = np.linalg.svd(df_adj, full_matrices=True)
print(U.shape,s.shape,V.shape)
print(s)
print(np.cumsum(s)/np.sum(s))

In [ ]:
plt.plot(range(1,1+s.size),np.cumsum(s)/np.sum(s),'o-')
#plt.plot([0,2],[0.85,0.85])
plt.ylabel("Fraction of Variance Explained ($m_\ell$)")
plt.xlabel("Number of Singular Values ($\ell$)")
show("varexp")

In [ ]:
plt.plot(times*10,-1*V[0,:],'o-',label="$\\mathbf{v}_1$")
plt.plot(times*10,V[1,:],'*-',label="$\\mathbf{v}_2$")
#plt.plot(times,-V[2,:],'^-',label="Third Component")
plt.xlabel("time (ns)")
plt.ylabel("Value")
#plt.title("Singular Component Vectors of Data")
plt.legend(loc="best")
show("singcomp")

In [ ]:
def get_dante(i,U,s,V):
    S = np.zeros((s.size,s.size))
    np.fill_diagonal(S,s)
    tmp = np.dot(U[i,:],np.dot(S,V) )
    return tmp
print(np.argmax(U[:,0]),settings[np.argmax(U[:,0])])
print(np.argmin(U[:,0]),settings[np.argmin(U[:,0])])
print(np.argmax(U[:,1]),settings[np.argmax(U[:,1])])
print(np.argmin(U[:,1]),settings[np.argmin(U[:,1])])

In [ ]:
#compare predictions 
i = 1
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),'r--',label="1 singular value")
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),':',color="brown",label="2 singular value")
plt.plot(plt_times*10,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5)
plt.annotate('Case 1', xy=(1.25,np.max(df[i,:])+0.0025))
i = 4
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),'r--')
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),':',color="brown")
plt.plot(plt_times*10,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5)
plt.annotate('Case 4', xy=(1.25,np.max(df[i,:])-0.02))
plt.plot(plt_times*10, interpdata(times,mean_trace,plt_times), "b-.",label="mean response")
plt.legend(loc="best")
plt.xlabel("time (ns)")
plt.ylabel("$T$ (keV)")
#plt.title("Actual and Model Dante Temperatures varying Scale")
show("scale")

In [ ]:
#compare predictions 
i = 5
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),'r--',label="1 singular value")
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),':',color="brown",label="2 singular value")
plt.plot(plt_times*10,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5)
plt.annotate('Case 5', xy=(1.25,np.max(df[i,:])-0.03))
i = 9
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),'r--')
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,tmp+mean_trace,plt_times),':',color="brown")
plt.plot(plt_times*10,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5)
plt.annotate('Case 9', xy=(1.25,np.max(df[i,:])+0.005))
plt.legend(loc="best")
plt.xlabel("time (ns)")
plt.ylabel("$T$ (keV)")
#plt.title("Actual and Model Dante Temperatures varying Scale")
show("scale_2")

In [ ]:
#compare predictions 
i = 1
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times,interpdata(times,tmp+mean_trace,plt_times),'--',label="1 singular value")
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times,interpdata(times,tmp+mean_trace,plt_times),':',label="2 singular value")
plt.plot(plt_times,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5)
i = 4
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times,interpdata(times,tmp+mean_trace,plt_times),'--')
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times,interpdata(times,tmp+mean_trace,plt_times),':')
plt.plot(plt_times,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5)
plt.legend(loc="best")
plt.xlabel("time (sh)")
plt.ylabel("$T_\mathrm{rad}$ (keV)")
#plt.title("Actual and Model Dante Temperatures varying Scale")
show("scale")

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
y = -U[:,0]
X = np.zeros((y.shape[0],4))
X[:,0] = settings[:,0]
X[:,1] = settings[:,0]**2
X[:,2] = settings[:,1]
X[:,3] = settings[:,2]**2
reg = LinearRegression(fit_intercept=True).fit(X, y)

y2 = U[:,1]
X2 = settings.copy()
reg2 = LinearRegression(fit_intercept=True).fit(X2, y2)

In [ ]:
print(reg.coef_)
print(reg.intercept_)
print(reg.score(X,y))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.score(X2,y2))

In [ ]:
tmp = alldata[ (alldata[:,3]==0.8)*(alldata[:,4]==1)*(alldata[:,5]==1),:]
plt.plot(tmp[:,0]*10,tmp[:,1],'r*',label="simulation data")
tmpU = np.zeros((1,2))
tmpU[0,0] = -reg.predict(np.array([0.8,0.8**2,1,1**2]).reshape(1,4))
tmpU[0,1] = reg2.predict(np.array([0.8,1,1]).reshape((1,3)))
print(tmpU.shape)
tmp = get_dante(0,tmpU,s[0:2],V[0:2,:])
plt.plot(times*10,tmp+mean_trace,'--',label="linear model")
i = 1
l = 1
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
l = 2
tmp = get_dante(i,U[:,0:l],s[0:l],V[0:l,:])
plt.plot(plt_times*10,interpdata(times,df[i,:],plt_times),'k-',alpha=0.5,label="scale=0.85, scl=1, rapt=1")
plt.legend(loc="best")
plt.xlabel("time (ns)")
plt.ylabel("$T_\mathrm{rad}$ (keV)")
show("pred_sc8")